In [1]:
import os
import sys
import pathlib
from glob import glob 
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
import json

In [2]:
import numpy as np
import pandas as pd

In [3]:
import xarray as xr
import salem

In [4]:
salem.__version__

'0.2.3-5-g7864474'

### local function definition 

In [5]:
def make_ts(trmm_ts, climo): 
    
    trmm_ts_c = trmm_ts.copy()
    
    dec = ["{}%".format(x) for x in range(20, 100, 20)]
    
    categories = []
    anoms = []
    anoms_pc = []

    for i, row in trmm_ts_c.iterrows(): 

        month = row.name.month 

        val = row.values[0]

        clim = climo.loc[month, :]

        deciles = clim.loc[dec,]

        ave = clim.loc['mean']

        anom = val - ave

        anom_pc = (val / ave) * 100

        if (val < deciles.loc['20%']): 
            category = 'Well below'
        elif (deciles.loc['20%'] <= val <= deciles.loc['40%']): 
            category = 'Below'
        elif (deciles.loc['40%'] < val <= deciles.loc['60%']): 
            category = 'Normal'
        elif (deciles.loc['60%'] < val <= deciles.loc['80%']): 
            category = 'Above'    
        elif (val > deciles.loc['80%']): 
            category = 'Well above' 
        else: 
            print("category cannot be calculated")

        categories.append(category)

        anoms.append(anom)

        anoms_pc.append(anom_pc)
        
    trmm_ts_c.loc[:,'year'] = trmm_ts.index.year 

    trmm_ts_c.loc[:,'month'] = trmm_ts.index.month 

    trmm_ts_c.loc[:,'anomaly'] = np.array(anoms)

    trmm_ts_c.loc[:,'percent'] = np.array(anoms_pc)

    trmm_ts_c.loc[:,'category'] = np.array(categories)
    
    return trmm_ts_c.loc[:,['year','month','trmm','anomaly','percent','category']]

### defines the path to the climatologies 

In [6]:
dpath_climo = pathlib.Path.cwd().parent / 'outputs' / 'climatologies'

In [7]:
dpath_climo

PosixPath('/home/nicolasf/operational/ICU/ops/TRMM/outputs/climatologies')

### list the files (monthly climatologies for each Island Group)

In [8]:
lfiles_climo = list(dpath_climo.glob("monthly*.csv"))

### output path for the last 6 months time-series 

In [9]:
output_path = pathlib.Path.cwd().parent / 'outputs' / 'Time_Series' / 'last_6_months'

In [10]:
output_path

PosixPath('/home/nicolasf/operational/ICU/ops/TRMM/outputs/Time_Series/last_6_months')

### path to the updated **extended South Pacific** daily TRMM / GPM files 

In [11]:
dpath_TRMM = pathlib.Path.cwd().parents[1] / 'data' / 'TRMM' / 'daily' / 'extended_SP'

In [12]:
dpath_TRMM

PosixPath('/home/nicolasf/operational/ICU/ops/data/TRMM/daily/extended_SP')

In [13]:
lfiles_TRMM = list(dpath_TRMM.glob('*.nc'))

In [14]:
lfiles_TRMM.sort()

### check that the TRMM dataset has been updated to the latest available date here (~ 2 days lag to real time)

In [15]:
lfiles_TRMM[-10:]

[PosixPath('/home/nicolasf/operational/ICU/ops/data/TRMM/daily/extended_SP/3B42RT_daily.2019.03.14.nc'),
 PosixPath('/home/nicolasf/operational/ICU/ops/data/TRMM/daily/extended_SP/3B42RT_daily.2019.03.15.nc'),
 PosixPath('/home/nicolasf/operational/ICU/ops/data/TRMM/daily/extended_SP/3B42RT_daily.2019.03.16.nc'),
 PosixPath('/home/nicolasf/operational/ICU/ops/data/TRMM/daily/extended_SP/3B42RT_daily.2019.03.17.nc'),
 PosixPath('/home/nicolasf/operational/ICU/ops/data/TRMM/daily/extended_SP/3B42RT_daily.2019.03.18.nc'),
 PosixPath('/home/nicolasf/operational/ICU/ops/data/TRMM/daily/extended_SP/3B42RT_daily.2019.03.19.nc'),
 PosixPath('/home/nicolasf/operational/ICU/ops/data/TRMM/daily/extended_SP/3B42RT_daily.2019.03.20.nc'),
 PosixPath('/home/nicolasf/operational/ICU/ops/data/TRMM/daily/extended_SP/3B42RT_daily.2019.03.21.nc'),
 PosixPath('/home/nicolasf/operational/ICU/ops/data/TRMM/daily/extended_SP/3B42RT_daily.2019.03.22.nc'),
 PosixPath('/home/nicolasf/operational/ICU/ops/data/TRM

if not updated:

  
1) cd into `~/operational/ICU/ops/data/TRMM/daily/extended_SP/` and note the date of the last netcdf file   
2) activate the `pangeo` environment and cd into `~/operational/ICU/ops/data/TRMM/daily/scripts/`  
3) run:   

```
python get_daily_TRMM_TMPA_netcdf.py -o ../extended_SP -lonW 125 -lonE 240 -latN 25 -latS -50 -s YYYMMDD -e YYYYMMDD
```

Then re-run the lines above to get the updated list of files 

### get the last dates and determine the period to load in 

In [16]:
last_TRMM_file = lfiles_TRMM[-1]

In [17]:
dates_elems = list(map(int, str(last_TRMM_file).split('.')[-4:-1]))

In [18]:
last_date_TRMM = datetime(*dates_elems)

In [19]:
last_date_TRMM

datetime.datetime(2019, 3, 23, 0, 0)

### we want the last 6 months of daily TRMM data 

In [20]:
nmonths = 6

In [21]:
start_date_TRMM = datetime((last_date_TRMM - relativedelta(months=nmonths - 1)).year, (last_date_TRMM - relativedelta(months=nmonths - 1)).month, 1)

In [22]:
start_date_TRMM

datetime.datetime(2018, 10, 1, 0, 0)

### sanity check on the dates here

In [23]:
print("the last 6 months TRMM period will cover the days from {:%Y-%m-%d} to {:%Y-%m-%d}".format(start_date_TRMM, last_date_TRMM))

the last 6 months TRMM period will cover the days from 2018-10-01 to 2019-03-23


### now construct the final list of files (using pandas date_range function)

In [24]:
dates_TRMM = pd.date_range(start=start_date_TRMM, end=last_date_TRMM, freq='1D')

In [25]:
dates_TRMM

DatetimeIndex(['2018-10-01', '2018-10-02', '2018-10-03', '2018-10-04',
               '2018-10-05', '2018-10-06', '2018-10-07', '2018-10-08',
               '2018-10-09', '2018-10-10',
               ...
               '2019-03-14', '2019-03-15', '2019-03-16', '2019-03-17',
               '2019-03-18', '2019-03-19', '2019-03-20', '2019-03-21',
               '2019-03-22', '2019-03-23'],
              dtype='datetime64[ns]', length=174, freq='D')

In [26]:
lfiles_TRMM = [] 
for d in dates_TRMM: 
    lfiles_TRMM.append(dpath_TRMM / '3B42RT_daily.{:%Y.%m.%d}.nc'.format(d))

### and now loads the whole dataset of the last 6 months

In [27]:
dset = xr.open_mfdataset(lfiles_TRMM)

In [28]:
dset

<xarray.Dataset>
Dimensions:  (lat: 300, lon: 460, time: 174)
Coordinates:
  * lat      (lat) float32 -49.875 -49.625 -49.375 ... 24.375 24.625 24.875
  * lon      (lon) float32 125.125 125.375 125.625 ... 239.375 239.625 239.875
  * time     (time) datetime64[ns] 2018-10-01 2018-10-02 ... 2019-03-23
Data variables:
    trmm     (time, lat, lon) float32 dask.array<shape=(174, 300, 460), chunksize=(1, 300, 460)>

### calculates the monthly averages, will be preliminary for the last month if not all days downloaded

In [29]:
dset = dset.resample(time='1M').mean('time')

In [31]:
dset.compute()

<xarray.Dataset>
Dimensions:  (lat: 300, lon: 460, time: 6)
Coordinates:
  * time     (time) datetime64[ns] 2018-10-31 2018-11-30 ... 2019-03-31
  * lat      (lat) float32 -49.875 -49.625 -49.375 ... 24.375 24.625 24.875
  * lon      (lon) float32 125.125 125.375 125.625 ... 239.375 239.625 239.875
Data variables:
    trmm     (time, lat, lon) float32 2.4841936 2.6641932 ... 0.06521738

### Now loads the shapefiles 

In [32]:
shapes_ipath = pathlib.Path.cwd().parents[1] / 'data' / 'shapefiles' / 'ICU' / 'clipped' / 'countries_converted'

In [33]:
shapes_ipath

PosixPath('/home/nicolasf/operational/ICU/ops/data/shapefiles/ICU/clipped/countries_converted')

In [34]:
lshapefiles = shapes_ipath.glob('*/*.shp')

In [35]:
lshapefiles = list(lshapefiles)

In [36]:
lshapefiles.sort()

### open the dictionnay mapping country name in filenames to actual country name 

In [37]:
with open(pathlib.Path.cwd().parents[1] / 'resources' / 'dict_countries.json', 'r') as fj: 
    dict_countries = json.load(fj)

### Now loops over each shapefile, use the country geometry to clip and mask the TRMM dataset, calculates the regional average, and make the last 6 months Time-Series

In [38]:
for shp_filename in lshapefiles: 
    
    shapes = salem.read_shapefile(shp_filename)
    
    country_fname = os.path.basename(shp_filename)[7:-4]
    
    country_name = dict_countries[country_fname]
    
    print("processing {}".format(country_name))
    
    subset = dset.salem.subset(shape=shapes, margin=2)
    
    roi = subset.salem.roi(shape=shapes, all_touched=True)

    trmm_ts = roi.mean(dim=('lat','lon')).to_dataframe()
    
    last_date = trmm_ts.index[-1]
    
    climo = pd.read_csv(dpath_climo / 'monthly_climo_{}.csv'.format(country_fname), index_col=0)
    
    last_6_months = make_ts(trmm_ts, climo)
        
    last_6_months.to_csv(output_path / '{}_pbased.csv'.format(country_fname))

processing American Samoa
processing Austral Islands
processing Federated States of Micronesia
processing Fiji
processing Guam
processing Kiribati: Gilbert Islands
processing Kiribati: Line Islands
processing Kiribati: Phoenix Islands
processing Marquesas
processing Marshall Islands
processing Nauru
processing New Caledonia
processing Niue
processing Northern Cook Islands
processing Northern Marianas
processing Palau
processing Papua New Guinea
processing Pitcairn Islands
processing Samoa
processing Society Islands
processing Solomon Islands
processing Southern Cook Islands
processing Tokelau
processing Tonga
processing Tuamotu / Gambier Islands
processing Tuvalu
processing Vanuatu North
processing Vanuatu South
processing Wallis & Futuna


In [39]:
dpath_climo

PosixPath('/home/nicolasf/operational/ICU/ops/TRMM/outputs/climatologies')

In [40]:
output_path

PosixPath('/home/nicolasf/operational/ICU/ops/TRMM/outputs/Time_Series/last_6_months')